Following the hugging face quicktour at https://huggingface.co/docs/diffusers/quicktour.

Before starting, install the requirements via pip: `pip3 install -r requirements.txt`

# Load the model...

In [17]:
from diffusers import DiffusionPipeline
from pprint import pprint

pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensor=True)
# pprint(pipeline)

Keyword arguments {'use_safetensor': True} are not expected by StableDiffusionPipeline and will be ignored.
Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 12.93it/s]


# Setup for CPU

In [18]:
pipe = pipeline.to("cpu")

# Generate from prompt

In [4]:
prompt = "skinny black and white cat in ralph steadman style"
image = pipe(prompt).images[0]
image.save("generated.png")

  0%|          | 0/50 [00:00<?, ?it/s]

# Trying different schedulers

## EulerDiscreteScheduler

In [3]:
from diffusers import DiffusionPipeline, EulerDiscreteScheduler

pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensors=True)
pipeline.scheduler = EulerDiscreteScheduler.from_config(pipeline.scheduler.config)
pipe = pipeline.to("cpu")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]